# Importing Required Libraries

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install contractions
# apt install python3-contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 15.4 MB/s eta 0:00:00


In [3]:
!pip install scikit-learn

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from keras.layers import Dense
import json
import re
import csv
import string
import json

from sklearn.feature_extraction.text import TfidfVectorizer
import unicodedata
from sklearn.model_selection import train_test_split


import contractions
#

# Read a dialog file and create a DataFrame of questions and answers.

In [29]:
question  =[]
answer = []
# Opening JSON file
f = open('/content/drive/MyDrive/data.json')

data = json.load(f)

i=0
for i in range(len(data)):

    for j in data[i]["Q&As"]:
        question.append(list(j.values())[0])
        answer.append(list(j.values())[1])
data = pd.DataFrame({"question" : question ,"answer":answer})
data.head()


,question,answer
0,Why is it important to support BIPOC owned fas...,Supporting BIPOC owned fashion brands is impor...
1,Why is it challenging to find BIPOC owned fash...,It is challenging to find BIPOC owned fashion ...
2,What is the significance of featuring BIPOC ow...,Featuring BIPOC owned brands in content is sig...
3,How can supporting BIPOC owned brands contribu...,Supporting BIPOC owned brands is a tangible wa...
4,What is the message conveyed by the unique col...,The unique collections of BIPOC owned brands c...


In [30]:
# print(data[0]["Q&As"][0][0])
print(data["question"])

0       Why is it important to support BIPOC owned fas...
1       Why is it challenging to find BIPOC owned fash...
2       What is the significance of featuring BIPOC ow...
3       How can supporting BIPOC owned brands contribu...
4       What is the message conveyed by the unique col...
                              ...                        
3940    What is the significance of voting with your d...
3941    What inspired Rachel Temko to start Whimsy + Row?
3942    How does Whimsy + Row reduce its carbon footpr...
3943    What are some of the ethical practices followe...
3944    How does Whimsy + Row contribute to the commun...
Name: question, Length: 3945, dtype: object


# Preprocessing text data using unicode normalization, regular expressions, and string manipulation

In [40]:
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
      if unicodedata.category(c) != 'Mn')

def clean_text(text):
    text = unicode_to_ascii(text.lower().strip())
    text = " ".join(text.split())
    text = ''.join([i for i in text if not i.isdigit()])
    contractions.fix(text)
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"hi", "hey", text)
    text = re.sub(r"\r", "", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "what is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub("(\\W)"," ",text)
    text = re.sub('\S*\d\S*\s*','', text)
    text =  "<sos> " +  text + " <eos>"
    return text

#

# Clean, Convert, & Tokenize the data

In [41]:
data["question"] = data.question.apply(clean_text)
data["answer"] = data.answer.apply(clean_text)
question  = data.question.values.tolist()
answer =  data.answer.values.tolist()

def tokenize(lang):
    lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
    lang_tokenizer.fit_on_texts(lang)
    tensor = lang_tokenizer.texts_to_sequences(lang)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')
    return tensor, lang_tokenizer
print(len(data["question"]))

3945


#

# Setting up data for sequence-to-sequence model training!
`Here, we perform various data preprocessing steps like tokenization, removal of start and end tags, and creation of training and validation datasets using an 80-20 split. The tokenization process involves creating word-to-index and index-to-word dictionaries for both input and target languages. We can also set various hyperparameters for the model like the embedding dimension, number of units, vocabulary size, batch size, and buffer size.`

In [42]:
input_tensor , inp_lang  =  tokenize(question)
target_tensor , targ_lang  =  tokenize(answer)

def remove_tags(sentence):
    return sentence.split("<start>")[-1].split("<end>")[0]

max_length_targ, max_length_inp = target_tensor.shape[1], input_tensor.shape[1]

# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 100
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim =256
units = 1024
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

print((input_tensor[0][3]) , inp_lang ,target_tensor , targ_lang)

28 <keras.src.preprocessing.text.Tokenizer object at 0x78a33648baf0> [[   3  116 1702 ...    0    0    0]
 [   3   20    9 ...    0    0    0]
 [   3 1248 1702 ...    0    0    0]
 ...
 [   3  750  746 ...    0    0    0]
 [   3  750  746 ...    0    0    0]
 [   3  750  746 ...    0    0    0]] <keras.src.preprocessing.text.Tokenizer object at 0x78a072fde0e0>


In [44]:
data["answer"][1000]

'<sos> consumers should ensure that the hemp fabric they are purchasing is organic and produced using environmentally friendly processes some companies use harmful chemicals in the production of hemp fabric so it is important to research the brand before making a purchase <eos>'

In [45]:
# for i in target_tensor :
#     print(i)
print(input_tensor[0])
print(len(target_tensor))

[  1  22   5  28  44   9  80 741 742  11  17   2   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0]
3945


# Encoder class for Sequence-to-Sequence model

In [46]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.enc_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')

    def call(self, x,hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state = hidden)
        return output, state

    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))

#

# Encoder Model Initialization and Output Shape

In [47]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder output shape: (batch size, sequence length, units) (100, 30, 1024)
Encoder Hidden state shape: (batch size, units) (100, 1024)


#

# Bahdanau Attention Layer Implementation for Neural Machine Translation
`The code defines a custom Keras layer that implements Bahdanau attention mechanism for Neural Machine Translation. Bahdanau attention is a type of attention mechanism that computes attention weights over the input sequence at every time step, and generates a context vector that is a weighted sum of the input sequence. This context vector is then used by the decoder to generate the output sequence. The implementation uses two dense layers, W1 and W2, to map the input and query to a common hidden space, and a dense layer V to generate the attention weights. The layer takes as input the query (decoder hidden state) and the values (encoder output sequence), and outputs the context vector and attention weights.`

In [48]:
class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, query, values):
        # query hidden state shape == (batch_size, hidden size)
        # query_with_time_axis shape == (batch_size, 1, hidden size)
        # values shape == (batch_size, max_len, hidden size)
        # we are doing this to broadcast addition along the time axis to calculate the score
        query_with_time_axis = tf.expand_dims(query, 1)

        # score shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying score to self.V
        # the shape of the tensor before applying self.V is (batch_size, max_length, units)
        score = self.V(tf.nn.tanh(
            self.W1(query_with_time_axis) + self.W2(values)))

        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)

#         print("attention_weights : ",attention_weights)

        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights

#

# Applying Bahdanau Attention to Encoder Outputs

In [49]:
attention_layer = BahdanauAttention(10)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

Attention result shape: (batch size, units) (100, 1024)
Attention weights shape: (batch_size, sequence_length, 1) (100, 30, 1)


#

# A decoder class that takes the output from the encoder and generates the target sequence word by word!
`The Decoder class is a sub-class of the tf.keras.Model class, which defines the decoding part of the Sequence-to-Sequence model used for the chatbot. The decoder is responsible for generating the output sequence given the encoded input sequence and the previous state.`

In [50]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.dec_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')
        self.fc = tf.keras.layers.Dense(vocab_size)

        # used for attention
        self.attention = BahdanauAttention(self.dec_units)

    def call(self, x, hidden, enc_output):
        # enc_output shape == (batch_size, max_length, hidden_size)
        context_vector, attention_weights = self.attention(hidden, enc_output)

        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        x = self.embedding(x)

        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

        # passing the concatenated vector to the GRU
        output, state = self.gru(x)

        # output shape == (batch_size * 1, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))

        # output shape == (batch_size, vocab)
        x = self.fc(output)

        return x, state, attention_weights

#

# Decoder model initialization and sample output

In [51]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

sample_decoder_output, _, _ = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                      sample_hidden, sample_output)

print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

Decoder output shape: (batch_size, vocab size) (100, 8650)


#

# Loss function using Adam optimizer

In [52]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)
#     print("real : ",real,"\npredicted : ",pred)
#     print ("loss_ : ",loss_)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

#

# Training Step Function using Teacher Forcing

In [53]:
@tf.function
def train_step(inp, targ, enc_hidden,variables=np.array([]),gradients=np.array([])):#,variables=np.array([])
    loss = 0

    with tf.GradientTape() as tape:
        enc_output, enc_hidden = encoder(inp, enc_hidden)

        dec_hidden = enc_hidden

        dec_input = tf.expand_dims([targ_lang.word_index['<sos>']] * BATCH_SIZE, 1)

        # Teacher forcing - feeding the target as the next input
        for t in range(1, targ.shape[1]):
            # passing enc_output to the decoder
            predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

            loss += loss_function(targ[:, t], predictions)
#             print("loss : ",loss)
            # using teacher forcing
            dec_input = tf.expand_dims(targ[:, t], 1)

    batch_loss = (loss / int(targ.shape[1]))

#     if len(variables)<=0:
    variables = encoder.trainable_variables + decoder.trainable_variables
#     print("variables : ",variables)
#     print("vrb : ",vrb)
    gradients = tape.gradient(loss, variables)

#     print("gradients : ",gradients)

    optimizer.apply_gradients(zip(gradients, variables))

    return batch_loss, variables,gradients

#

# Training loop for the Chatbot Model
`It appears that the loss decreases with each epoch, which suggests that the model is improving and learning to generate more accurate translations. The decreasing loss values are also a good sign that the optimizer and loss function are appropriate for the task at hand.`

In [ ]:
# from tempfile import TemporaryFile
# outfile = TemporaryFile()
# np.save(outfile,vrb[0].numpy)

In [ ]:
# if you are running first time then ignore running this cell
vrb=grd=[]
with open('/content/drive/MyDrive/vrb.npy', 'rb') as f:
    ind=0
    for i in f:
        vrb.append(np.load(f,allow_pickle=True))
        ind+=1
with open('/content/drive/MyDrive/grd.npy', 'rb') as f:
    ind=0
    for i in f:
        grd.append(np.load(f,allow_pickle=True))
        ind+=1

In [54]:
EPOCHS = 50
vrb=grd=[]
for epoch in range(1, EPOCHS + 1):
    print(epoch)
    enc_hidden = encoder.initialize_hidden_state()
    total_loss = 0

    for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
        batch_loss,vrb,grd = train_step(inp, targ, enc_hidden,vrb,grd)#,vrb,grd
        total_loss += batch_loss

with open('/content/drive/MyDrive/vrb.npy', 'wb') as f:
    for i in range(len(vrb)):
        np.save(f, vrb[i].numpy)
    print("grd\n")
with open('/content/drive/MyDrive/grd.npy', 'wb') as f:
    for i in range(len(grd)):
        print(i)
        if(i==0 or i==4):
            np.save(f,grd[i].indices.numpy())
            np.save(f,grd[i].values.numpy())
        else:
            np.save(f,grd[i].numpy)
#         np.save(f, )

#     if(epoch % 2 == 0):
    print('Epoch:{:3d} Loss:{:.4f}'.format(epoch,
                                          total_loss / steps_per_epoch))


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
grd

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
Epoch: 50 Loss:0.1190


#

# Evaluation function for the Chatbot Model

In [31]:
def evaluate(sentence):
    sentence = clean_text(sentence)
    try:
        inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]
    except:
        return "I am sorry",sentence
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                         maxlen=max_length_inp,
                                                         padding='post')
    inputs = tf.convert_to_tensor(inputs)

    result = ''

    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang.word_index['<sos>']], 0)

    for t in range(max_length_targ):
        predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                             dec_hidden,
                                                             enc_out)

        # storing the attention weights to plot later on
        attention_weights = tf.reshape(attention_weights, (-1, ))

        predicted_id = tf.argmax(predictions[0]).numpy()

        result += targ_lang.index_word[predicted_id] + ' '

        if targ_lang.index_word[predicted_id] == '<eos>':
            return remove_tags(result), remove_tags(sentence)

        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)

    return remove_tags(result), remove_tags(sentence)

#

# Loading and using our Chatbot model to answer questions!

In [56]:
def ask(sentence):
    result, sentence = evaluate(sentence)

    print('Question: %s' % (sentence))
    print('Predicted answer: {}'.format(result))
    return result
ask(questions[36])
print("Actual : ")
print(questions[36],"\n",answers[36])

IndexError: list index out of range

In [ ]:
# #ask(questions[5])
# print(questions[:11])

In [35]:
ask('What is BIPOC?')

Question: <sos> what is bipoc <eos>
Predicted answer: chanels impact on the fasheyon industry is the unjustified and unsustainable speed in the fasheyon industry is the unjustified and unsustainable speed in the fasheyon industry is the unjustified and unsustainable speed in the fasheyon industry is the unjustified and unsustainable speed in the fasheyon industry is the unjustified and unsustainable speed in the fasheyon industry is the unjustified and unsustainable speed in the fasheyon industry is the unjustified and unsustainable speed in the fasheyon industry is the unjustified and unsustainable speed in the fasheyon industry 


'chanels impact on the fasheyon industry is the unjustified and unsustainable speed in the fasheyon industry is the unjustified and unsustainable speed in the fasheyon industry is the unjustified and unsustainable speed in the fasheyon industry is the unjustified and unsustainable speed in the fasheyon industry is the unjustified and unsustainable speed in the fasheyon industry is the unjustified and unsustainable speed in the fasheyon industry is the unjustified and unsustainable speed in the fasheyon industry is the unjustified and unsustainable speed in the fasheyon industry '

In [36]:
import tkinter as tk

def submit_question():
    question = entry.get()
    answer = chatbot(question)
    text_area.insert(tk.END, "You: " + question + "\n")
    text_area.insert(tk.END, "Chatbot: " + answer + "\n\n")
    entry.delete(0, tk.END)  # Clear the entry field after submitting question

def chatbot(question):
    # Replace this with your chatbot logic
    return ask(question)

# Create the main window
root = tk.Tk()
root.title("Chatbot Interface")

# Create text area for displaying chat history
text_area = tk.Text(root, width=50, height=20)
text_area.pack(pady=10)

# Create entry field for inputting questions
entry = tk.Entry(root, width=40)
entry.pack(pady=5)

# Create submit button
submit_button = tk.Button(root, text="Submit", command=submit_question)
submit_button.pack(pady=5)

# Run the main event loop
root.mainloop()

TclError: no display name and no $DISPLAY environment variable

#

## Thank you for your time!